In [2]:
'/Users/macbook/Downloads/exports_10-07-2025 3/10-07-2025_ranking_2025-05-01 00:00:00_2025-05-01 00:00:00.xlsx'
import pandas as pd

# File paths
ranking_file = '/Users/macbook/Downloads/exports_10-07-2025 3/10-07-2025_ranking_2025-05-01 00:00:00_2025-05-01 00:00:00.xlsx'
profile_file = '/Users/macbook/Downloads/exports_10-07-2025 3/10-07-2025_profile_2025-05-01 00:00:00_2025-05-01 00:00:00.xlsx'

# Read Excel files
df_ranking = pd.read_excel(ranking_file)
df_profile = pd.read_excel(profile_file)

# Combine on 'tool_id'
df_merged = pd.merge(df_ranking, df_profile, on='tool_id', how='inner')

In [3]:
# Keep only the specified columns: tool_id, use_original, use aggreagted, name_tool
# Note: column names are case-sensitive and may need to be adjusted if they differ in the actual DataFrame
columns_to_keep = ['tool_id', 'use_original', 'use_aggregated', 'name_tool', 'tool_url_y', 'tool_description']
df_selected = df_merged[columns_to_keep]
df_selected.head()


,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description
0,744,;prototype development;ui/ux design & optimiza...,prototype development;ui/ux design & optimizat...,Figma,figma.com,Figma is a collaborative design platform enabl...
1,12052,;data analysis;data visualization;data governance,;data analysis;data visualization;data governance,DataBricks,databricks.com,Databricks offers an integrated platform for d...
2,12057,;foundation models (llm),foundation models (llm),Grok,x.ai,Grok-2 offers advanced conversational AI and p...
3,6660,;foundation models (llm),foundation models (llm),ChatGPT,chatgpt.com,Multimodal chat service exposes large language...
4,778,;payroll management;compliance management;cont...,payroll management;compliance management;contr...,Deel,deel.com,Deel provides a comprehensive platform for glo...


In [3]:
# Count how many tool names end with 'ai' (case-insensitive)
num_tools_end_with_ai = df_merged['name_tool'].dropna().apply(lambda x: str(x).strip().lower().endswith('ai')).sum()
print(f"Number of tools whose name ends with 'ai': {num_tools_end_with_ai}")


Number of tools whose name ends with 'ai': 3645


In [1]:
df_merged.head()

NameError: name 'df_merged' is not defined

In [5]:
# Keep only the specified columns: tool_id, use_original, use aggreagted, name_tool
# Note: column names are case-sensitive and may need to be adjusted if they differ in the actual DataFrame
columns_to_keep = ['tool_id', 'use_original', 'use_aggregated', 'name_tool', 'tool_url_y', 'tool_description']
df_selected = df_merged[columns_to_keep]
df_selected.head()


,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description
0,744,;prototype development;ui/ux design & optimiza...,prototype development;ui/ux design & optimizat...,Figma,figma.com,Figma is a collaborative design platform enabl...
1,12052,;data analysis;data visualization;data governance,;data analysis;data visualization;data governance,DataBricks,databricks.com,Databricks offers an integrated platform for d...
2,12057,;foundation models (llm),foundation models (llm),Grok,x.ai,Grok-2 offers advanced conversational AI and p...
3,6660,;foundation models (llm),foundation models (llm),ChatGPT,chatgpt.com,Multimodal chat service exposes large language...
4,778,;payroll management;compliance management;cont...,payroll management;compliance management;contr...,Deel,deel.com,Deel provides a comprehensive platform for glo...


In [4]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19238 entries, 0 to 19237
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           19238 non-null  int64 
 1   use_original      11537 non-null  object
 2   use_aggregated    11339 non-null  object
 3   name_tool         19238 non-null  object
 4   tool_url_y        19238 non-null  object
 5   tool_description  13568 non-null  object
dtypes: int64(1), object(5)
memory usage: 901.9+ KB


In [140]:
# Find a couple of rows where 'use_original' and 'use_aggregated' are different
diff_rows = df_selected[df_selected['use_original'] != df_selected['use_aggregated']]
# Display the first few such rows
diff_rows.sample(5)


,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description
13988,16588,NaN,NaN,IELTS Podcast,essaycheck.ieltspodcast.co,NaN
10570,100,NaN,NaN,Altera AI,altera.ai,Altera creates digital human beings with AI ar...
13499,22566,NaN,NaN,FILL GENIUS,fillgenius.com,FILL GENIUS automates the process of completin...
14703,20021,NaN,NaN,StoryDiffusion AI,storydiffusion.org,NaN
4745,21453,NaN,NaN,The Fabricant,thefabricant.ai,"Digital‑fashion house creating 3D garments, NF..."


In [4]:

from typing import List, Optional, Union
import pandas as pd

def filter_by_keywords(
    df: pd.DataFrame,
    column: str,
    keywords: List[str],
    case_sensitive: bool = False
) -> pd.DataFrame:
    """
    Filter DataFrame rows where the specified column contains any of the given keywords.
    """
    def contains_keywords(text):
        if not isinstance(text, str):
            return False
        text_cmp = text if case_sensitive else text.lower()
        kw_list = keywords if case_sensitive else [kw.lower() for kw in keywords]
        return any(kw in text_cmp for kw in kw_list)
    return df[df[column].apply(contains_keywords)]

def remove_keywords_from_column(
    df: pd.DataFrame,
    column: str,
    keywords: List[str],
    tool_ids: Optional[List[int]] = None
) -> pd.DataFrame:
    """
    Remove specified keywords from a column, optionally only for certain tool_ids.
    """
    def remove_keywords(text):
        if not isinstance(text, str):
            return text
        result = text
        for kw in keywords:
            # Remove keyword with or without leading/trailing semicolons and extra spaces
            result = result.replace(f";{kw}", "")
            result = result.replace(f"{kw};", "")
            result = result.replace(kw, "")
        # Clean up any accidental double semicolons or leading/trailing semicolons
        result = result.replace(";;", ";").strip(";").strip()
        return result

    mask = df['tool_id'].isin(tool_ids) if tool_ids is not None else pd.Series([True]*len(df), index=df.index)
    df.loc[mask, column] = df.loc[mask, column].apply(remove_keywords)
    return df

def add_missing_tool_ids(
    filtered_df: pd.DataFrame,
    df_selected: pd.DataFrame,
    tool_ids_to_add: List[int]
) -> pd.DataFrame:
    """
    Add rows from df_selected to filtered_df for tool_ids not already present.
    """
    existing_tool_ids = set(filtered_df['tool_id'])
    missing_tool_ids = [tid for tid in tool_ids_to_add if tid not in existing_tool_ids]
    missing_rows = df_selected[df_selected['tool_id'].isin(missing_tool_ids)]
    return pd.concat([filtered_df, missing_rows], ignore_index=True)


### Software development

In [44]:
# Find all rows where 'use_original' contains any of the specified keywords
keywords = [
    'app development',
    'software development',
    'website development',
]

def contains_keywords_use_original(text):
    text = str(text).lower()
    return any(keyword in text for keyword in keywords)

# Filter the DataFrame for relevant rows
filtered_df = df_selected[df_selected['use_original'].apply(contains_keywords_use_original)]




In [45]:
# Find all rows where 'tool_description' contains 'software development' (case-insensitive)
desc_mask = df_selected['tool_description'].astype(str).str.lower().str.contains('app development', na=False)
desc_software_dev_df = df_selected[desc_mask]

# Display the first few such rows
desc_software_dev_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 37 entries, 166 to 17122
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           37 non-null     int64 
 1   use_original      33 non-null     object
 2   use_aggregated    33 non-null     object
 3   name_tool         37 non-null     object
 4   tool_url_x        37 non-null     object
 5   tool_description  37 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.0+ KB


In [46]:
# List of tool_ids to add if not already in filtered_df
tool_ids_to_add = [
    13653, 13360, 6052, 15514, 5467, 13189, 11563, 4853, 8901, 5013, 9134, 1843,
    10731, 19342, 6126, 1385, 469, 19680, 11257, 1474, 8419
]

# Find which tool_ids are not already in filtered_df
existing_tool_ids = set(filtered_df['tool_id'])
missing_tool_ids = [tid for tid in tool_ids_to_add if tid not in existing_tool_ids]

# Get the rows from df_selected for these missing tool_ids
missing_rows = df_selected[df_selected['tool_id'].isin(missing_tool_ids)]

# Append these rows to filtered_df
filtered_df = pd.concat([filtered_df, missing_rows], ignore_index=True)



In [47]:
# Save filtered_df to Excel file
desc_software_dev_df.to_excel('filtered_software_development_tools.xlsx', index=False)


In [48]:
filtered_df.sample(5)
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           299 non-null    int64 
 1   use_original      297 non-null    object
 2   use_aggregated    297 non-null    object
 3   name_tool         299 non-null    object
 4   tool_url_x        299 non-null    object
 5   tool_description  298 non-null    object
dtypes: int64(1), object(5)
memory usage: 14.1+ KB


In [49]:

# Add 'software development' to use_aggregated for these tools, if not already present
def add_software_development(agg):
    agg = str(agg) if pd.notnull(agg) else ''
    # Remove leading/trailing semicolons and whitespace
    agg_clean = agg.strip(';').strip()
    # Check if 'software development' is already present (case-insensitive)
    if 'software development' in agg_clean.lower().split(';'):
        return agg_clean
    # Add 'software development' at the end, separated by semicolon if needed
    if agg_clean:
        return agg_clean + ';software development'
    else:
        return 'software development'

# Update the original df_selected DataFrame in place
for idx in filtered_df.index:
    current_agg = df_selected.at[idx, 'use_aggregated']
    df_selected.at[idx, 'use_aggregated'] = add_software_development(current_agg)

filtered_df = df_selected.loc[filtered_df.index]  # Refresh filtered_df with updated values

filtered_df.head()


,tool_id,use_original,use_aggregated,name_tool,tool_url_x,tool_description
0,744,;prototype development;ui/ux design & optimiza...,prototype development;ui/ux design & optimizat...,Figma,https://figma.com,Figma is a collaborative design platform enabl...
1,12052,;data analysis;data visualization;data governance,data analysis;data visualization;data governan...,DataBricks,https://databricks.com,Databricks offers an integrated platform for d...
2,12057,;foundation models (llm),foundation models (llm);software development,Grok,https://x.ai,Grok-2 offers advanced conversational AI and p...
3,6660,;foundation models (llm),foundation models (llm);software development,ChatGPT,https://chatgpt.com,Multimodal chat service exposes large language...
4,778,;payroll management;compliance management;cont...,payroll management;compliance management;contr...,Deel,https://deel.com,Deel provides a comprehensive platform for glo...


In [50]:
# Save filtered_df to Excel with 'tool_url_x' renamed to 'tool_url'
filtered_df_to_save = filtered_df.rename(columns={'tool_url_x': 'tool_url'})
filtered_df_to_save.to_excel('software_development_sub_ranking.xlsx', index=False)


### Energy management, monitoring & analysis

In [71]:
# Find all rows where 'use_original' contains any of the specified keywords
keywords = [
"gaming"
]

def contains_keywords_use_original(text):
    text = str(text).lower()
    return any(keyword in text for keyword in keywords)

# Filter the DataFrame for relevant rows
filtered_df_energy = df_selected[df_selected['use_original'].apply(contains_keywords_use_original)]

# Save filtered_df to Excel file

In [72]:
filtered_df_energy.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 1017 to 19234
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           34 non-null     int64 
 1   use_original      34 non-null     object
 2   use_aggregated    34 non-null     object
 3   name_tool         34 non-null     object
 4   tool_url_x        34 non-null     object
 5   tool_description  34 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.9+ KB


In [55]:
# List of tool_ids for which to delete the specified keywords
tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]
keywords_to_remove = [
    "energy monitoring & analysis", "energy management", "climate control"
]

def remove_keywords_from_aggregated(text):
    if not isinstance(text, str):
        return text
    result = text
    for kw in keywords_to_remove:
        # Remove keyword with or without leading/trailing semicolons and extra spaces
        result = result.replace(f";{kw}", "")
        result = result.replace(f"{kw};", "")
        result = result.replace(kw, "")
    # Clean up any accidental double semicolons or leading/trailing semicolons
    result = result.replace(";;", ";").strip(";").strip()
    return result

mask = filtered_df_energy['tool_id'].isin(tool_ids_to_edit)
filtered_df_energy.loc[mask, 'use_aggregated'] = (
    filtered_df_energy.loc[mask, 'use_aggregated'].apply(remove_keywords_from_aggregated)
)



In [56]:
filtered_df_energy.head()


,tool_id,use_original,use_aggregated,name_tool,tool_url_x,tool_description
1704,12745,;energy optimization;carbon footprint analysis...,;energy optimization;carbon footprint analysis...,metron,https://metron.energy,The METRON Energy Management and Optimization ...
2232,13057,;energy management;heating control,;energy management;heating control,Mysa,https://getmysa.com,Mysa Smart Thermostats offer control over elec...
2894,3340,;transaction monitoring;app development;energy...,transaction monitoring;app development;pricing...,MeetFrank,https://meetfrank.com,"A comprehensive platform offering remote, hybr..."
3982,5511,;energy optimization;compliance management;ene...,;energy optimization;compliance management;ene...,Ecoplanet,https://ecoplanet.tech,Ecoplanet offers an advanced energy management...
5857,4653,;energy monitoring & analysis;energy management,;energy monitoring & analysis;energy management,etalytics,https://etalytics.com,etalytics provides AI-driven energy management...


In [57]:
filtered_df_energy.info()


<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 1704 to 19180
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           20 non-null     int64 
 1   use_original      20 non-null     object
 2   use_aggregated    20 non-null     object
 3   name_tool         20 non-null     object
 4   tool_url_x        20 non-null     object
 5   tool_description  20 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.1+ KB


In [54]:
#to excel
filtered_df_energy.to_excel('filtered_energy_tools.xlsx', index=False)

In [61]:
# Find all rows where 'tool_description' contains 'software development' (case-insensitive)
desc_mask = df_selected['tool_description'].astype(str).str.lower().str.contains('energy management', na=False)
energy_df = df_selected[desc_mask]

# Display the first few such rows
energy_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 13 entries, 1704 to 19180
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           13 non-null     int64 
 1   use_original      10 non-null     object
 2   use_aggregated    10 non-null     object
 3   name_tool         13 non-null     object
 4   tool_url_x        13 non-null     object
 5   tool_description  13 non-null     object
dtypes: int64(1), object(5)
memory usage: 728.0+ bytes


In [ ]:
# Save filtered_df to Excel file
desc_software_dev_df.to_excel('filtered_software_development_tools.xlsx', index=False)

In [ ]:
# List of tool_ids to add if not already in filtered_df
tool_ids_to_add = [
    13653, 13360, 6052, 15514, 5467, 13189, 11563, 4853, 8901, 5013, 9134, 1843,
    10731, 19342, 6126, 1385, 469, 19680, 11257, 1474, 8419
]

# Find which tool_ids are not already in filtered_df
existing_tool_ids = set(filtered_df['tool_id'])
missing_tool_ids = [tid for tid in tool_ids_to_add if tid not in existing_tool_ids]

# Get the rows from df_selected for these missing tool_ids
missing_rows = df_selected[df_selected['tool_id'].isin(missing_tool_ids)]

# Append these rows to filtered_df
filtered_df = pd.concat([filtered_df, missing_rows], ignore_index=True)



## Gaming


In [141]:

from typing import List, Optional, Union
import pandas as pd

def filter_by_keywords(
    df: pd.DataFrame,
    column: str,
    keywords: List[str],
    case_sensitive: bool = False
) -> pd.DataFrame:
    """
    Filter DataFrame rows where the specified column contains any of the given keywords.
    """
    def contains_keywords(text):
        if not isinstance(text, str):
            return False
        text_cmp = text if case_sensitive else text.lower()
        kw_list = keywords if case_sensitive else [kw.lower() for kw in keywords]
        return any(kw in text_cmp for kw in kw_list)
    return df[df[column].apply(contains_keywords)]

def remove_keywords_from_column(
    df: pd.DataFrame,
    column: str,
    keywords: List[str],
    tool_ids: Optional[List[int]] = None
) -> pd.DataFrame:
    """
    Remove specified keywords from a column, optionally only for certain tool_ids.
    """
    def remove_keywords(text):
        if not isinstance(text, str):
            return text
        result = text
        for kw in keywords:
            # Remove keyword with or without leading/trailing semicolons and extra spaces
            result = result.replace(f";{kw}", "")
            result = result.replace(f"{kw};", "")
            result = result.replace(kw, "")
        # Clean up any accidental double semicolons or leading/trailing semicolons
        result = result.replace(";;", ";").strip(";").strip()
        return result

    mask = df['tool_id'].isin(tool_ids) if tool_ids is not None else pd.Series([True]*len(df), index=df.index)
    df.loc[mask, column] = df.loc[mask, column].apply(remove_keywords)
    return df

def add_missing_tool_ids(
    filtered_df: pd.DataFrame,
    df_selected: pd.DataFrame,
    tool_ids_to_add: List[int]
) -> pd.DataFrame:
    """
    Add rows from df_selected to filtered_df for tool_ids not already present.
    """
    existing_tool_ids = set(filtered_df['tool_id'])
    missing_tool_ids = [tid for tid in tool_ids_to_add if tid not in existing_tool_ids]
    missing_rows = df_selected[df_selected['tool_id'].isin(missing_tool_ids)]
    return pd.concat([filtered_df, missing_rows], ignore_index=True)


In [151]:

# Define your keywords and tool_ids for any use case
keywords = [
    "gaming"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids
#keywords_to_remove = keywords  # Can be different if needed

# 1. Filter by keywords in 'use_original'
filtered_df_gaming = filter_by_keywords(df_selected, 'use_original', keywords)

In [152]:
filtered_df_gaming.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 1017 to 19234
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           34 non-null     int64 
 1   use_original      34 non-null     object
 2   use_aggregated    34 non-null     object
 3   name_tool         34 non-null     object
 4   tool_url_y        34 non-null     object
 5   tool_description  34 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.9+ KB


In [153]:
# 2. Remove keywords from 'use_aggregated' for certain tool_ids
filtered_df_gaming = remove_keywords_from_column(filtered_df_gaming, 'use_aggregated', keywords_to_remove, tool_ids=tool_ids_to_edit)

In [146]:

# 3. Save to Excel (filename can be parameterized)
filtered_df_gaming.to_excel('filtered_tools.xlsx', index=False)


In [147]:

# 4. Filter by keywords in 'tool_description' (example: for "software development" or any other)
desc_keywords = ["gaming"]  # Replace with e.g. ["software development", "gaming"] as needed
desc_df = filter_by_keywords(df_selected, 'tool_description', desc_keywords)
desc_df.info()


# 3. Save to Excel (filename can be parameterized)
desc_df.to_excel('filtered_tools2.xlsx', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 45 entries, 47 to 19181
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           45 non-null     int64 
 1   use_original      31 non-null     object
 2   use_aggregated    31 non-null     object
 3   name_tool         45 non-null     object
 4   tool_url_y        45 non-null     object
 5   tool_description  45 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.5+ KB


In [148]:
# 5. Add missing tool_ids to filtered_df (if needed)
tool_ids_to_add = [
    13653, 13360, 6052, 15514, 5467, 13189, 11563, 4853, 8901, 5013, 9134, 1843,
    10731, 19342, 6126, 1385, 469, 19680, 11257, 1474, 8419
]
filtered_df_gaming = add_missing_tool_ids(filtered_df_gaming, df_selected, tool_ids_to_add)


In [150]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           55 non-null     int64 
 1   use_original      53 non-null     object
 2   use_aggregated    53 non-null     object
 3   name_tool         55 non-null     object
 4   tool_url_y        55 non-null     object
 5   tool_description  55 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.7+ KB


In [138]:
filtered_df.to_excel('gaming_sub_ranking.xlsx', index=False)

software development,


energy monitoring & analysis, energy management, climate control

gaming

fashion recommendation

food & drinks recommendations

receipt management

drone management

disease detection, medical image analysis

fall prevention

recruiting & hiring

legal assistance

collision detection & avoidance ( https://www.spoor.ai/collision-detection)

damage detection

car damage detection

travel planning

# ai ethics and governance



In [76]:

# Define your keywords and tool_ids for any use case
keywords = [
    "fall prevention"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids
#keywords_to_remove = keywords  # Can be different if needed

# 1. Filter by keywords in 'use_original'
filtered_df = filter_by_keywords(df_selected, 'use_original', keywords)

## Medical image & disease detection

In [102]:
# Define your keywords and tool_ids for any use case
keywords = [
    "medical image analysis", "disease detection"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
tool_ids_to_edit = [6772, 3068, 10408, 13407, 8413, 7127, 1384, 1395, 24355, 6363]  # Example tool_ids
keywords_to_remove = keywords  # Can be different if needed

# 1. Filter by keywords in 'use_original'
filtered_df = filter_by_keywords(df_selected, 'use_original', keywords)

In [105]:
# 2. Remove keywords from 'use_aggregated' for certain tool_ids
filtered_df = remove_keywords_from_column(filtered_df, 'use_aggregated', keywords_to_remove, tool_ids=tool_ids_to_edit)

In [110]:
# Replace 'disease detection' with 'medical disease detection' in 'use_aggregated' column of filtered_df
filtered_df['use_aggregated'] = filtered_df['use_aggregated'].str.replace(
    'disease detection', 'medical disease detection', regex=False
)


/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_59086/2578463729.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['use_aggregated'] = filtered_df['use_aggregated'].str.replace(


In [119]:
# 4. Filter by keywords in 'tool_description' that are NOT in filtered_df
desc_keywords = ["disease"]  # Replace with e.g. ["software development", "gaming"] as needed

# Get all rows in df_selected where 'tool_description' contains any of desc_keywords
desc_df_all = filter_by_keywords(df_selected, 'tool_description', desc_keywords)

# Exclude those that are already in filtered_df (by tool_id)
desc_df = desc_df_all[~desc_df_all['tool_id'].isin(filtered_df['tool_id'])]

desc_df.info()

# 3. Save to Excel (filename can be parameterized)
desc_df.to_excel('filtered_tools3_disease.xlsx', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 354 to 19050
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           24 non-null     int64 
 1   use_original      17 non-null     object
 2   use_aggregated    17 non-null     object
 3   name_tool         24 non-null     object
 4   tool_url_x        24 non-null     object
 5   tool_description  24 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.3+ KB


In [115]:
desc_df.head()


,tool_id,use_original,use_aggregated,name_tool,tool_url_x,tool_description
383,6775,;digital & ai-driven therapy;therapy managemen...,digital & ai-driven therapy;therapy management...,Insitro,https://insitro.com,insitro employs machine learning to integrate ...
408,6639,;disease detection;clinical trial management,;medical disease detection;clinical trial mana...,InSilico Medicine,https://insilico.com,PHARMA.AI integrates cutting-edge generative A...
412,6726,;disease detection;disease treatment;patient c...,;medical disease detection;disease treatment;p...,Viz.ai,https://viz.ai,Viz.ai offers an FDA-cleared AI-powered care c...
3687,6772,;climate & weather analysis;disease detection;...,climate & weather analysis;farm management & o...,Wolkus HQ,https://fasal.co,Fasal leverages IoT and AI to offer data-drive...
4777,3068,;disease detection;climate & weather analysis;...,climate & weather analysis;satellite image ana...,Agrio,https://agrio.app,Agrio is an app that uses AI algorithms to pro...


In [108]:
filtered_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 42 entries, 383 to 19104
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           42 non-null     int64 
 1   use_original      42 non-null     object
 2   use_aggregated    42 non-null     object
 3   name_tool         42 non-null     object
 4   tool_url_x        42 non-null     object
 5   tool_description  41 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.3+ KB


In [111]:
filtered_df.head()

,tool_id,use_original,use_aggregated,name_tool,tool_url_x,tool_description
383,6775,;digital & ai-driven therapy;therapy managemen...,digital & ai-driven therapy;therapy management...,Insitro,https://insitro.com,insitro employs machine learning to integrate ...
408,6639,;disease detection;clinical trial management,;medical disease detection;clinical trial mana...,InSilico Medicine,https://insilico.com,PHARMA.AI integrates cutting-edge generative A...
412,6726,;disease detection;disease treatment;patient c...,;medical disease detection;disease treatment;p...,Viz.ai,https://viz.ai,Viz.ai offers an FDA-cleared AI-powered care c...
1109,13571,;disease detection;deep learning,;medical disease detection;deep learning,PMcardio,https://powerfulmedical.com,PMcardio's AI-powered platform enables rapid c...
1329,12910,;speech recognition;natural language processin...,;speech recognition;natural language processin...,Defined.ai,https://defined.ai,Defined.ai delivers ethically sourced and high...


In [112]:
# Save the filtered DataFrame to an Excel file
filtered_df.to_excel("disease.xlsx", index=False)

In [91]:

# Define your keywords and tool_ids for any use case
keywords = [
    "medical image analysis", "disease detection"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids
#keywords_to_remove = keywords  # Can be different if needed

# 1. Filter by keywords in 'use_original'
filtered_df = filter_by_keywords(df_selected, 'use_original', keywords)

## Background Removal 

In [ ]:

# Define your keywords and tool_ids for any use case
keywords = [
    "background removal"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids

# 1. Filter by keywords in 'use_original'
bg_removal = filter_by_keywords(df_selected, 'use_original', keywords)

In [16]:
bg_removal.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 176 entries, 0 to 175
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           176 non-null    int64 
 1   use_original      173 non-null    object
 2   use_aggregated    170 non-null    object
 3   name_tool         176 non-null    object
 4   tool_url_y        176 non-null    object
 5   tool_description  176 non-null    object
dtypes: int64(1), object(5)
memory usage: 8.4+ KB


In [9]:
bg_removal.head()

,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description
54,13666,;image generation;background removal;logo desi...,;image generation;background removal;logo desi...,Kittl,kittl.com,Kittl is a browser-based design tool offering ...
143,18802,;image resizing;image color editing;background...,;image editing;image color editing;background ...,PicLumen,piclumen.com,PicLumen is an AI-driven image creation platfo...
152,5684,;image generation;background removal;image edi...,;image generation;background removal;image edi...,Recraft,recraft.ai,Recraft offers a premium image generation and ...
203,13467,;background removal,background removal,Remove.bg,remove.bg,remove.bg automatically removes image backgrou...
241,12663,;icon generation;image resizing;background rem...,;icon generation;image editing;background remo...,Icons8,icons8.com,"Icons8 provides apps and tools for design, off..."


In [13]:
# 4. Filter by keywords in 'tool_description' that are NOT in filtered_df
desc_keywords = ["background removal"]  # Replace with e.g. ["software development", "gaming"] as needed

# Get all rows in df_selected where 'tool_description' contains any of desc_keywords
desc_df_all = filter_by_keywords(df_selected, 'tool_description', desc_keywords)

# Exclude those that are already in filtered_df (by tool_id)
desc_df = desc_df_all[~desc_df_all['tool_id'].isin(bg_removal['tool_id'])]

desc_df.info()

# 3. Save to Excel (filename can be parameterized)
desc_df.to_excel('background_removal.xlsx', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 3 entries, 250 to 4418
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           3 non-null      int64 
 1   use_original      0 non-null      object
 2   use_aggregated    0 non-null      object
 3   name_tool         3 non-null      object
 4   tool_url_y        3 non-null      object
 5   tool_description  3 non-null      object
dtypes: int64(1), object(5)
memory usage: 168.0+ bytes


In [ ]:
tool_ids_to_add = [21308, 5342, 18266]  # Example tool_ids
# 5. Add missing tool_ids to filtered_df (if needed)

bg_removal = add_missing_tool_ids(bg_removal, df_selected, tool_ids_to_add)

In [43]:
import os, json, openai
from pydantic import BaseModel, ValidationError

openai.api_key = os.getenv("OPENAI_API_KEY")       # keep your key outside code

# 1️⃣  Pydantic schema
class BGCheck(BaseModel):
    background_removal: bool
    reason: str | None = None

# 2️⃣  Structured‑output request
SYSTEM = "You are a JSON‑only API. Respond via the function call."
USER_TMPL = "Does this tool do *background removal*?\n\n\"\"\"{desc}\"\"\""

def check_bg_removal(desc: str) -> BGCheck:
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": USER_TMPL.format(desc=desc)}
        ],
        functions=[{
            "name": "bg_check",
            "description": "Return whether tool performs background removal",
            "parameters": BGCheck.model_json_schema()   #  ← auto‑build JSON Schema
        }],
        function_call={"name": "bg_check"}              #  ← ask for that output
    )

    args_json = completion.choices[0].message.function_call.arguments
    try:
        return BGCheck(**json.loads(args_json))
    except (ValidationError, json.JSONDecodeError) as e:
        raise RuntimeError(f"Invalid LLM output: {e}")

# --- quick demo ---------------------------
if __name__ == "__main__":
    sample = "Online editor that instantly removes image backgrounds."
    result = check_bg_removal(sample)
    print(result.model_dump())        # {'background_removal': True, 'reason': None}


{'background_removal': True, 'reason': 'Online editor that instantly removes image backgrounds.'}


In [18]:
# ⬇️ assumes you already executed the earlier cell that defines `check_bg_removal`
import pandas as pd

# df is the DataFrame you showed:
# Columns: tool_id, use_original, use_aggregated, name_tool, tool_url_y, tool_description

# 1. Call the LLM once per row and get a dict back
results = (
    bg_removal["tool_description"]
    .apply(lambda desc: check_bg_removal(desc).model_dump())  # → {"background_removal":…, "reason": …}
)

# 2. Expand that dict into two columns and attach them in one line
bg_removal[["background_removal", "reason"]] = results.apply(pd.Series)

# Optional: inspect
print(bg_removal.head())


   tool_id                                       use_original  \
0    13666  ;image generation;background removal;logo desi...   
1    18802  ;image resizing;image color editing;background...   
2     5684  ;image generation;background removal;image edi...   
3    13467                                ;background removal   
4    12663  ;icon generation;image resizing;background rem...   

                                      use_aggregated  name_tool    tool_url_y  \
0  ;image generation;background removal;logo desi...      Kittl     kittl.com   
1  ;image editing;image color editing;background ...   PicLumen  piclumen.com   
2  ;image generation;background removal;image edi...    Recraft    recraft.ai   
3                                 background removal  Remove.bg     remove.bg   
4  ;icon generation;image editing;background remo...     Icons8    icons8.com   

                                    tool_description  background_removal  \
0  Kittl is a browser-based design tool offeri

In [20]:
TARGET_USE = "background removal"

def _to_list(cell: str) -> list[str]:
    """Convert a ;‑separated string (or NaN) → list without blanks."""
    if pd.isna(cell):
        return []
    return [u.strip() for u in str(cell).split(";") if u.strip()]

def update_use_aggregated(row):
    uses = _to_list(row["use_aggregated"])

    if row["background_removal"]:
        if TARGET_USE not in uses:
            uses.append(TARGET_USE)
    else:
        uses = [u for u in uses if u != TARGET_USE]

    # Keep the original style: leading ';' if the cell started with one
    prefix = ";" if str(row["use_aggregated"]).startswith(";") else ""
    return prefix + ";".join(uses)

bg_removal["use_aggregated"] = bg_removal.apply(update_use_aggregated, axis=1)


In [46]:
SEARCH_SYSTEM = (
    "You are an analyst with access to a web_search tool. "
    "Search only enough to decide whether the given tool really performs "
    "background removal. Return a JSON dict with keys:\n"
    "  supports  – bool\n"
    "  evidence  – one‑sentence citation‑ready summary"
)

def verify_with_web(tool_name: str, tool_url: str) -> tuple[bool, str]:
    """Ask the search‑preview model for confirmation/evidence."""
    completion = openai.chat.completions.create(
        model="gpt-4o-mini-search-preview",
        temperature=0,
        tools=[{"type": "web_search"}],   # built‑in tool
        #tool_choice="auto",
        messages=[
            {"role": "system", "content": SEARCH_SYSTEM},
            {"role": "user",
             "content": (
                 f"Tool name: {tool_name}\n"
                 f"Homepage: {tool_url}\n\n"
                 "Decide if it offers *image background removal*. "
                 "Use web_search if needed. "
                 "Respond with JSON {{\"supports\": <bool>, \"evidence\": \"...\"}}."
             )
            }
        ],
    )

    resp = completion.choices[0].message.content
    data = json.loads(resp)
    return bool(data["supports"]), str(data["evidence"])


In [47]:
def robust_bg_check(row) -> BGCheck:
    """Run the 1st‑pass LLM; if it says False, verify with search."""
    first = check_bg_removal(row["tool_description"])

    if first.background_removal:
        return first            # already positive ⇒ trust it

    # First pass said “False” – verify
    supports, evidence = verify_with_web(row["name_tool"], row["tool_url_y"])

    if supports:
        return BGCheck(
            background_removal=True,
            reason=evidence,     # web evidence
            verified=True
        )
    else:
        # keep original “False” but mark that we actually checked
        first.verified = True
        first.reason = (first.reason or "") + " (double‑checked on the web)"
        return first


In [48]:
# Call once per row, just like before
results = (
    bg_removal
    .apply(robust_bg_check, axis=1)
    .apply(lambda r: r.model_dump())        # → dict
)

bg_removal[["background_removal", "reason", "verified"]] = results.apply(pd.Series)


BadRequestError: Error code: 400 - {'error': {'message': "Missing required parameter: 'tools[0].function'.", 'type': 'invalid_request_error', 'param': 'tools[0].function', 'code': 'missing_required_parameter'}}

In [22]:
bg_removal.to_excel("bg_removal.xlsx", index=False)

In [21]:
bg_removal.head()


,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description,background_removal,reason
0,13666,;image generation;background removal;logo desi...,;image generation;logo design;text writing;ima...,Kittl,kittl.com,Kittl is a browser-based design tool offering ...,False,The description does not mention background re...
1,18802,;image resizing;image color editing;background...,;image editing;image color editing;character d...,PicLumen,piclumen.com,PicLumen is an AI-driven image creation platfo...,False,The description does not mention background re...
2,5684,;image generation;background removal;image edi...,;image generation;image editing;mockup generation,Recraft,recraft.ai,Recraft offers a premium image generation and ...,False,The description does not mention background re...
3,13467,;background removal,background removal,Remove.bg,remove.bg,remove.bg automatically removes image backgrou...,True,The description indicates that remove.bg autom...
4,12663,;icon generation;image resizing;background rem...,;icon generation;image editing;face swap;profi...,Icons8,icons8.com,"Icons8 provides apps and tools for design, off...",False,The description does not mention background re...


In [ ]:
## Background Removal 

## Validating autonomous vehicles & transport

In [ ]:
# Define your keywords and tool_ids for any use case
keywords = [
    "autonomous vehicles & transport"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids

# 1. Filter by keywords in 'use_original'
auto_vehicles = filter_by_keywords(df_selected, 'use_aggregated', keywords)

In [35]:
auto_vehicles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 125 to 18558
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           20 non-null     int64 
 1   use_original      20 non-null     object
 2   use_aggregated    20 non-null     object
 3   name_tool         20 non-null     object
 4   tool_url_y        20 non-null     object
 5   tool_description  20 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.1+ KB


In [ ]:
# 4. Filter by keywords in 'tool_description' that are NOT in filtered_df
desc_keywords = ["background removal"]  # Replace with e.g. ["software development", "gaming"] as needed

# Get all rows in df_selected where 'tool_description' contains any of desc_keywords
desc_df_all = filter_by_keywords(df_selected, 'tool_description', desc_keywords)

# Exclude those that are already in filtered_df (by tool_id)
desc_df = desc_df_all[~desc_df_all['tool_id'].isin(bg_removal['tool_id'])]

desc_df.info()

# 3. Save to Excel (filename can be parameterized)
desc_df.to_excel('background_removal.xlsx', index=False)


In [50]:
import os, json, openai
from pydantic import BaseModel, ValidationError

openai.api_key = os.getenv("OPENAI_API_KEY")       # keep your key outside code

# 1️⃣  Pydantic schema
class BGCheck(BaseModel):
    autonomous_vehicle_transport: bool
    reason: str | None = None

# 2️⃣  Structured‑output request
SYSTEM = "You are a JSON‑only API. Respond via the function call."
USER_TMPL = "Does this tool or company do *autonomous vehicle & transport*, or is very closely related to this use case or business?\n\n\"\"\"{desc}\"\"\""

def check_desc(desc: str) -> BGCheck:
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": USER_TMPL.format(desc=desc)}
        ],
        functions=[{
            "name": "autonomous",
            "description": "Return whether tool performs autonomous vehicle & transport",
            "parameters": BGCheck.model_json_schema()   #  ← auto‑build JSON Schema
        }],
        function_call={"name": "autonomous"}              #  ← ask for that output
    )

    args_json = completion.choices[0].message.function_call.arguments
    try:
        return BGCheck(**json.loads(args_json))
    except (ValidationError, json.JSONDecodeError) as e:
        raise RuntimeError(f"Invalid LLM output: {e}")

# --- quick demo ---------------------------
if __name__ == "__main__":
    sample = "Shield AI builds autonomous systems for drones and aircraft, focusing on the Hivemind AI pilot to enable operations without GPS or human pilots, providing innovative solutions for both military and commercial use."
    result = check_desc(sample)
    print(result.model_dump())        # {'background_removal': True, 'reason': None}


{'autonomous_vehicle_transport': True, 'reason': 'Shield AI develops autonomous systems for drones and aircraft, which are closely related to autonomous vehicle and transport technologies.'}


In [51]:
# ⬇️ assumes you already executed the earlier cell that defines `check_bg_removal`
import pandas as pd

# df is the DataFrame you showed:
# Columns: tool_id, use_original, use_aggregated, name_tool, tool_url_y, tool_description

# 1. Call the LLM once per row and get a dict back
results = (
    auto_vehicles["tool_description"]
    .apply(lambda desc: check_desc(desc).model_dump())  # → {"background_removal":…, "reason": …}
)

# 2. Expand that dict into two columns and attach them in one line
auto_vehicles[["autonomous_vehicle_transport", "reason"]] = results.apply(pd.Series)

# Optional: inspect
print(auto_vehicles.head())


      tool_id                                     use_original  \
125     12158                 ;autonomous vehicles & transport   
126     12140                 ;autonomous vehicles & transport   
136     12153                 ;autonomous vehicles & transport   
537      6721                                  ;model training   
1032    13067  ;predictive analytics;sensor fusion;ai copilots   

                                         use_aggregated  name_tool tool_url_y  \
125                     autonomous vehicles & transport    Pony.ai    pony.ai   
126                     autonomous vehicles & transport  Shield AI  shield.ai   
136                     autonomous vehicles & transport       Nuro    nuro.ai   
537      model training;autonomous vehicles & transport     PlusAI    plus.ai   
1032  predictive analytics;sensor fusion;ai copilots...    Helm.ai    helm.ai   

                                       tool_description  \
125   Pony.ai is a global leader in autonomous drivi...  

/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_50777/1554959920.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  auto_vehicles[["autonomous_vehicle_transport", "reason"]] = results.apply(pd.Series)


In [52]:
auto_vehicles.head()

,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description,autonomous_vehicle_transport,reason
125,12158,;autonomous vehicles & transport,autonomous vehicles & transport,Pony.ai,pony.ai,Pony.ai is a global leader in autonomous drivi...,True,Pony.ai specializes in autonomous driving tech...
126,12140,;autonomous vehicles & transport,autonomous vehicles & transport,Shield AI,shield.ai,Shield AI builds autonomous systems for drones...,True,Shield AI develops autonomous systems for dron...
136,12153,;autonomous vehicles & transport,autonomous vehicles & transport,Nuro,nuro.ai,Nuro offers AI-first autonomous driving techno...,True,Nuro specializes in AI-first autonomous drivin...
537,6721,;model training,model training;autonomous vehicles & transport,PlusAI,plus.ai,Plus revolutionizes autonomous driving with a ...,True,The company focuses on autonomous driving and ...
1032,13067,;predictive analytics;sensor fusion;ai copilots,predictive analytics;sensor fusion;ai copilots...,Helm.ai,helm.ai,Helm.ai provides AI solutions for ADAS to L4 a...,True,Helm.ai provides AI solutions specifically for...


In [ ]:
TARGET_USE = "background removal"

def _to_list(cell: str) -> list[str]:
    """Convert a ;‑separated string (or NaN) → list without blanks."""
    if pd.isna(cell):
        return []
    return [u.strip() for u in str(cell).split(";") if u.strip()]

def update_use_aggregated(row):
    uses = _to_list(row["use_aggregated"])

    if row["background_removal"]:
        if TARGET_USE not in uses:
            uses.append(TARGET_USE)
    else:
        uses = [u for u in uses if u != TARGET_USE]

    # Keep the original style: leading ';' if the cell started with one
    prefix = ";" if str(row["use_aggregated"]).startswith(";") else ""
    return prefix + ";".join(uses)

bg_removal["use_aggregated"] = bg_removal.apply(update_use_aggregated, axis=1)


In [ ]:
bg_removal.to_excel("bg_removal.xlsx", index=False)

In [ ]:
class BGCheck(BaseModel):
    background_removal: bool
    reason: str | None = None
    verified: bool = False          # ← True = second‑pass verification ran


## Validating legal assictance

In [13]:

# Define your keywords and tool_ids for any use case
keywords = [
    "legal assistance"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids

# 1. Filter by keywords in 'use_original'
legal_assistance = filter_by_keywords(df_selected, 'use_aggregated', keywords)

# 2. Save to Excel


In [15]:
legal_assistance.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 158 to 18977
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           25 non-null     int64 
 1   use_original      24 non-null     object
 2   use_aggregated    25 non-null     object
 3   name_tool         25 non-null     object
 4   tool_url_y        25 non-null     object
 5   tool_description  24 non-null     object
dtypes: int64(1), object(5)
memory usage: 1.4+ KB


In [6]:
# 4. Filter by keywords in 'tool_description' that are NOT in filtered_df
desc_keywords = ["legal assistance"]  # Replace with e.g. ["software development", "gaming"] as needed

# Get all rows in df_selected where 'tool_description' contains any of desc_keywords
desc_df_all = filter_by_keywords(df_selected, 'tool_description', desc_keywords)

# Exclude those that are already in filtered_df (by tool_id)
desc_df = desc_df_all[~desc_df_all['tool_id'].isin(legal_assistance['tool_id'])]

desc_df.info()

# 3. Save to Excel (filename can be parameterized)
desc_df.to_excel('legal_assistance2.xlsx', index=False)


<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 11360 to 15440
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           4 non-null      int64 
 1   use_original      4 non-null      object
 2   use_aggregated    4 non-null      object
 3   name_tool         4 non-null      object
 4   tool_url_y        4 non-null      object
 5   tool_description  4 non-null      object
dtypes: int64(1), object(5)
memory usage: 224.0+ bytes


In [8]:
import os, json, openai
from pydantic import BaseModel, ValidationError

openai.api_key = os.getenv("OPENAI_API_KEY")       # keep your key outside code

# 1️⃣  Pydantic schema
class BGCheck(BaseModel):
    legal_assistance: bool
    reason: str | None = None

# 2️⃣  Structured‑output request
SYSTEM = "You are a JSON‑only API. Respond via the function call."
USER_TMPL = "Does this tool or company do *legal assistance*, or is very closely related to this use case or business?\n\n\"\"\"{desc}\"\"\""

def check_desc(desc: str) -> BGCheck:
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": USER_TMPL.format(desc=desc)}
        ],
        functions=[{
            "name": "legal_assistance",
            "description": "Return whether tool performs legal assistance",
            "parameters": BGCheck.model_json_schema()   #  ← auto‑build JSON Schema
        }],
        function_call={"name": "legal_assistance"}              #  ← ask for that output
    )

    args_json = completion.choices[0].message.function_call.arguments
    try:
        return BGCheck(**json.loads(args_json))
    except (ValidationError, json.JSONDecodeError) as e:
        raise RuntimeError(f"Invalid LLM output: {e}")

# --- quick demo ---------------------------
if __name__ == "__main__":
    sample = "DoNotPay is an AI-powered platform that helps users contest parking tickets, manage subscriptions, and access various legal services, making legal assistance more accessible and affordable."
    result = check_desc(sample)
    print(result.model_dump())        # {'legal_assistance': True, 'reason': ...}


{'legal_assistance': True, 'reason': 'The platform provides access to various legal services, including contesting parking tickets, which qualifies as legal assistance.'}


In [10]:
# ⬇️ assumes you already executed the earlier cell that defines `check_bg_removal`
import pandas as pd

# df is the DataFrame you showed:
# Columns: tool_id, use_original, use_aggregated, name_tool, tool_url_y, tool_description

# 1. Call the LLM once per row and get a dict back
results = (
    legal_assistance["tool_description"]
    .apply(lambda desc: check_desc(desc).model_dump())  # → {"background_removal":…, "reason": …}
)

# 2. Expand that dict into two columns and attach them in one line
legal_assistance[["legal_assistance", "reason"]] = results.apply(pd.Series)

# Optional: inspect
print(legal_assistance.head())


      tool_id                                       use_original  \
158     13774  ;(academic) literature review;document analysi...   
632       895                  ;legal assistance;data management   
1211    13144  ;legal assistance;email automation;invoice & p...   
1449     3962  ;document processing;contract review & analysi...   
1986    12957                                  ;legal assistance   

                                         use_aggregated          name_tool  \
158   ;(academic) literature review;document analysi...             Harvey   
632                   ;legal assistance;data management  Reveal-Brainspace   
1211  ;legal assistance;email marketing & automation...            Tonkean   
1449  ;document processing;contract review & analysi...          Spellbook   
1986                                  ;legal assistance  Onna Technologies   

           tool_url_y                                   tool_description  \
158         harvey.ai  Harvey provides a secur

/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/3022791949.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  legal_assistance[["legal_assistance", "reason"]] = results.apply(pd.Series)
/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/3022791949.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  legal_assistance[["legal_assistance", "reason"]] = results.apply(pd.Series)


In [ ]:
legal_assistance.head()

In [11]:
TARGET_USE = "legal assistance"

def _to_list(cell: str) -> list[str]:
    """Convert a ;‑separated string (or NaN) → list without blanks."""
    if pd.isna(cell):
        return []
    return [u.strip() for u in str(cell).split(";") if u.strip()]

def update_use_aggregated(row):
    uses = _to_list(row["use_aggregated"])

    if row["legal_assistance"]:
        if TARGET_USE not in uses:
            uses.append(TARGET_USE)
    else:
        uses = [u for u in uses if u != TARGET_USE]

    # Keep the original style: leading ';' if the cell started with one
    prefix = ";" if str(row["use_aggregated"]).startswith(";") else ""
    return prefix + ";".join(uses)

legal_assistance["use_aggregated"] = legal_assistance.apply(update_use_aggregated, axis=1)


/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/2136605413.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  legal_assistance["use_aggregated"] = legal_assistance.apply(update_use_aggregated, axis=1)


In [12]:
# Export the legal_assistance DataFrame to an Excel file
legal_assistance.to_excel("legal_assistance.xlsx", index=False)


## 

## Validating audio generation 

In [19]:
# Define your keywords and tool_ids for any use case
keywords = [
    "audio generation"
    # Add more as needed, e.g. "gaming", "software development", etc.
]
#tool_ids_to_edit = [3340, 13388, 3703, 11555, 14017, 5056]  # Example tool_ids

# 1. Filter by keywords in 'use_original'
audio = filter_by_keywords(df_selected, 'use_aggregated', keywords)

In [20]:
# 4. Filter by keywords in 'tool_description' that are NOT in filtered_df
desc_keywords = ["audio generation"]  # Replace with e.g. ["software development", "gaming"] as needed

# Get all rows in df_selected where 'tool_description' contains any of desc_keywords
desc_df_all = filter_by_keywords(df_selected, 'tool_description', desc_keywords)

# Exclude those that are already in filtered_df (by tool_id)
desc_df = desc_df_all[~desc_df_all['tool_id'].isin(audio['tool_id'])]

desc_df.info()

# 3. Save to Excel (filename can be parameterized)
desc_df.to_excel('audio_generation.xlsx', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 4 entries, 2479 to 12335
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           4 non-null      int64 
 1   use_original      4 non-null      object
 2   use_aggregated    4 non-null      object
 3   name_tool         4 non-null      object
 4   tool_url_y        4 non-null      object
 5   tool_description  4 non-null      object
dtypes: int64(1), object(5)
memory usage: 224.0+ bytes


In [21]:
import os, json, openai
from pydantic import BaseModel, ValidationError

openai.api_key = os.getenv("OPENAI_API_KEY")       # keep your key outside code

# 1️⃣  Pydantic schema
class BGCheck(BaseModel):
    audio: bool
    reason: str | None = None

# 2️⃣  Structured‑output request
SYSTEM = "You are a JSON‑only API. Respond via the function call."
USER_TMPL = "Does this tool or company do *audio generation*, or is very closely related to this use case or business?\n\n\"\"\"{desc}\"\"\""

def check_desc(desc: str) -> BGCheck:
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": USER_TMPL.format(desc=desc)}
        ],
        functions=[{
            "name": "audio",
            "description": "Return whether tool performs audio generation",
            "parameters": BGCheck.model_json_schema()   #  ← auto‑build JSON Schema
        }],
        function_call={"name": "audio"}              #  ← ask for that output
    )

    args_json = completion.choices[0].message.function_call.arguments
    try:
        return BGCheck(**json.loads(args_json))
    except (ValidationError, json.JSONDecodeError) as e:
        raise RuntimeError(f"Invalid LLM output: {e}")

# --- quick demo ---------------------------
if __name__ == "__main__":
    sample = "Shield AI builds autonomous systems for drones and aircraft, focusing on the Hivemind AI pilot to enable operations without GPS or human pilots, providing innovative solutions for both military and commercial use."
    result = check_desc(sample)
    print(result.model_dump())        # {'background_removal': True, 'reason': None}


{'audio': False, 'reason': 'The company focuses on autonomous systems for drones and aircraft, which does not involve audio generation.'}


In [22]:
# ⬇️ assumes you already executed the earlier cell that defines `check_bg_removal`
import pandas as pd

# df is the DataFrame you showed:
# Columns: tool_id, use_original, use_aggregated, name_tool, tool_url_y, tool_description

# 1. Call the LLM once per row and get a dict back
results = (
    audio["tool_description"]
    .apply(lambda desc: check_desc(desc).model_dump())  # → {"background_removal":…, "reason": …}
)

# 2. Expand that dict into two columns and attach them in one line
audio[["audio", "reason"]] = results.apply(pd.Series)

# Optional: inspect
print(audio.head())


     tool_id                                       use_original  \
11      6552            ;audio generation;music recommendations   
32      6760  ;video editing;avatar generation;audio generat...   
591     5842                                  ;audio generation   
659     7636      ;audio editing;text to audio;audio generation   
671     6080  ;video editing;caption generation;image manage...   

                                        use_aggregated       name_tool  \
11             ;audio generation;music recommendations  Epidemic Sound   
32   video editing;avatar generation;audio generati...       Synthesia   
591                                  ;audio generation     cartesia.ai   
659      ;audio editing;text to audio;audio generation         Replica   
671  ;video editing;caption generation;image manage...        Submagic   

             tool_url_y                                   tool_description  \
11    epidemicsound.com  Epidemic Sound offers a vast catalog of royalt...

/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/1075009728.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  audio[["audio", "reason"]] = results.apply(pd.Series)
/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/1075009728.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  audio[["audio", "reason"]] = results.apply(pd.Series)


In [ ]:
TARGET_USE = "audio generation"

def _to_list(cell: str) -> list[str]:
    """Convert a ;‑separated string (or NaN) → list without blanks."""
    if pd.isna(cell):
        return []
    return [u.strip() for u in str(cell).split(";") if u.strip()]

def update_use_aggregated(row):
    uses = _to_list(row["use_aggregated"])

    if row["audio"]:
        if TARGET_USE not in uses:
            uses.append(TARGET_USE)
    else:
        uses = [u for u in uses if u != TARGET_USE]

    # Keep the original style: leading ';' if the cell started with one
    prefix = ";" if str(row["use_aggregated"]).startswith(";") else ""
    return prefix + ";".join(uses)

audio["use_aggregated"] = audio.apply(update_use_aggregated, axis=1)


/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/4111307947.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  audio["use_aggregated"] = audio.apply(update_use_aggregated, axis=1)


In [30]:
audio.head()

,tool_id,use_original,use_aggregated,name_tool,tool_url_y,tool_description,audio,reason
11,6552,;audio generation;music recommendations,;audio generation;music recommendations,Epidemic Sound,epidemicsound.com,Epidemic Sound offers a vast catalog of royalt...,True,Epidemic Sound provides royalty-free music and...
32,6760,;video editing;avatar generation;audio generat...,video editing;avatar generation;audio generati...,Synthesia,synthesia.io,Synthesia provides an AI platform for creating...,True,"Synthesia's platform includes voiceovers, whic..."
591,5842,;audio generation,;audio generation,cartesia.ai,cartesia.ai,Cartesia offers real-time multimodal intellige...,True,The mention of generative voice APIs indicates...
659,7636,;audio editing;text to audio;audio generation,;audio editing;text to audio;audio generation,Replica,replicastudios.com,Replica Studios provides AI-driven voice gener...,True,Replica Studios provides AI-driven voice gener...
671,6080,;video editing;caption generation;image manage...,;video editing;caption generation;image manage...,Submagic,submagic.co,Submagic offers a streamlined video editing to...,False,Submagic focuses on video editing and content ...


In [31]:
# Export the `audio` DataFrame to an Excel file
audio.to_excel("audio_cleaned.xlsx", index=False)


## Validating interviews & interview management

In [ ]:
# --- Modular pipeline for any use case (example: interviews & interview management) ---

import os, json, openai
from pydantic import BaseModel, ValidationError
import pandas as pd

# === 1. CONFIGURATION: Set your use case here ===
USE_CASE = "interviews & interview management"
TARGET_USE = "interviews & interview management"
KEYWORDS = [
    "interview",
    "interview management",
    "candidate screening",
    "interview scheduling",
    "interview analytics",
    # Add more as needed
]
EXCEL_OUT = "interviews_management_cleaned.xlsx"

# === 2. Filter by keywords in 'use_aggregated' ===
def filter_by_keywords(df, col, keywords):
    """Return rows where any keyword appears in the given column (case-insensitive)."""
    pattern = "|".join([rf"\b{k}\b" for k in keywords])
    return df[df[col].str.contains(pattern, case=False, na=False)]

# Use your main DataFrame (replace as needed)
df_use = filter_by_keywords(df_selected, 'use_aggregated', KEYWORDS)

# === 3. Filter by keywords in 'tool_description' not already in df_use ===
desc_df_all = filter_by_keywords(df_selected, 'tool_description', KEYWORDS)
desc_df = desc_df_all[~desc_df_all['tool_id'].isin(df_use['tool_id'])]

# Optional: inspect
desc_df.info()

# === 4. Save to Excel (filename parameterized) ===
desc_df.to_excel(EXCEL_OUT, index=False)

# === 5. LLM schema and function for this use case ===
openai.api_key = os.getenv("OPENAI_API_KEY")       # keep your key outside code

class UseCaseCheck(BaseModel):
    relevant: bool
    reason: str | None = None

SYSTEM = "You are a JSON‑only API. Respond via the function call."
USER_TMPL = (
    f"Does this tool or company do *{USE_CASE}*, or is very closely related to this use case or business?\n\n"
    "\"\"\"{desc}\"\"\""
)

def check_desc(desc: str) -> UseCaseCheck:
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": USER_TMPL.format(desc=desc)}
        ],
        functions=[{
            "name": "relevant",
            "description": f"Return whether tool performs {USE_CASE}",
            "parameters": UseCaseCheck.model_json_schema()
        }],
        function_call={"name": "relevant"}
    )
    args_json = completion.choices[0].message.function_call.arguments
    try:
        return UseCaseCheck(**json.loads(args_json))
    except (ValidationError, json.JSONDecodeError) as e:
        raise RuntimeError(f"Invalid LLM output: {e}")

if __name__ == "__main__":
    sample = "HireVue provides a platform for digital interviewing, candidate screening, and interview analytics for HR teams."
    result = check_desc(sample)
    print(result.model_dump())

# === 6. Apply LLM to DataFrame ===
results = (
    df_use["tool_description"]
    .apply(lambda desc: check_desc(desc).model_dump())
)
df_use[["relevant", "reason"]] = results.apply(pd.Series)

# Optional: inspect
print(df_use.head())

# === 7. Update use_aggregated column for the use case ===
def _to_list(cell: str) -> list[str]:
    """Convert a ;‑separated string (or NaN) → list without blanks."""
    if pd.isna(cell):
        return []
    return [u.strip() for u in str(cell).split(";") if u.strip()]

def update_use_aggregated(row):
    uses = _to_list(row["use_aggregated"])
    if row["relevant"]:
        if TARGET_USE not in uses:
            uses.append(TARGET_USE)
    else:
        uses = [u for u in uses if u != TARGET_USE]
    prefix = ";" if str(row["use_aggregated"]).startswith(";") else ""
    return prefix + ";".join(uses)

df_use["use_aggregated"] = df_use.apply(update_use_aggregated, axis=1)

# Save the updated DataFrame
df_use.to_excel(EXCEL_OUT, index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 50 entries, 1978 to 18927
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           50 non-null     int64 
 1   use_original      40 non-null     object
 2   use_aggregated    36 non-null     object
 3   name_tool         50 non-null     object
 4   tool_url_y        50 non-null     object
 5   tool_description  50 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.7+ KB
{'relevant': True, 'reason': 'HireVue specializes in digital interviewing and candidate screening, which directly relates to interview management.'}
     tool_id                                       use_original  \
58       725  ;market research;competitor analysis;interview...   
72     12155  ;recruiting & hiring;crm solutions;conversatio...   
244    12659  ;interviews & interview management;learning an...   
376    13778           ;interview automation;payroll managemen

/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/839994927.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use[["relevant", "reason"]] = results.apply(pd.Series)
/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/839994927.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use[["relevant", "reason"]] = results.apply(pd.Series)
/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/839994927.py:106: SettingWithCopyWarning: 
A value is trying to be set on a

## Validating compliance management

In [33]:
# --- Modular pipeline for any use case (example: interviews & interview management) ---

import os, json, openai
from pydantic import BaseModel, ValidationError
import pandas as pd

# === 1. CONFIGURATION: Set your use case here ===
USE_CASE = "compliance management"
TARGET_USE = "compliance management"
KEYWORDS = [
    "compliance",
    "regulatory compliance",
    "regulatory management",
    
    "regulatory audit",

    "regulatory technology",
    "regtech",

    "risk & compliance",

    
    "legal compliance",
    "compliance automation",
    "compliance monitoring",
    "compliance reporting",
    "compliance risk",
    "compliance requirements",
    "compliance obligations",
    "compliance audit",

    
]
EXCEL_OUT = "compliance_management_cleaned.xlsx"

# === 2. Filter by keywords in 'use_aggregated' ===
def filter_by_keywords(df, col, keywords):
    """Return rows where any keyword appears in the given column (case-insensitive)."""
    pattern = "|".join([rf"\b{k}\b" for k in keywords])
    return df[df[col].str.contains(pattern, case=False, na=False)]

# Use your main DataFrame (replace as needed)
df_use = filter_by_keywords(df_selected, 'use_aggregated', KEYWORDS)

# === 3. Filter by keywords in 'tool_description' not already in df_use ===
desc_df_all = filter_by_keywords(df_selected, 'tool_description', KEYWORDS)
desc_df = desc_df_all[~desc_df_all['tool_id'].isin(df_use['tool_id'])]

# Optional: inspect
desc_df.info()

# === 4. Save to Excel (filename parameterized) ===
desc_df.to_excel(EXCEL_OUT, index=False)

# === 5. LLM schema and function for this use case ===
openai.api_key = os.getenv("OPENAI_API_KEY")       # keep your key outside code

class UseCaseCheck(BaseModel):
    relevant: bool
    reason: str | None = None

SYSTEM = "You are a JSON‑only API. Respond via the function call."
USER_TMPL = (
    f"Does this tool or company do *{USE_CASE}*, or is very closely related to this use case or business?\n\n"
    "\"\"\"{desc}\"\"\""
)

def check_desc(desc: str) -> UseCaseCheck:
    completion = openai.chat.completions.create(
        model="gpt-4o-mini",
        temperature=0,
        messages=[
            {"role": "system", "content": SYSTEM},
            {"role": "user", "content": USER_TMPL.format(desc=desc)}
        ],
        functions=[{
            "name": "relevant",
            "description": f"Return whether tool performs {USE_CASE}",
            "parameters": UseCaseCheck.model_json_schema()
        }],
        function_call={"name": "relevant"}
    )
    args_json = completion.choices[0].message.function_call.arguments
    try:
        return UseCaseCheck(**json.loads(args_json))
    except (ValidationError, json.JSONDecodeError) as e:
        raise RuntimeError(f"Invalid LLM output: {e}")

if __name__ == "__main__":
    sample = "HireVue provides a platform for digital interviewing, candidate screening, and interview analytics for HR teams."
    result = check_desc(sample)
    print(result.model_dump())

# === 6. Apply LLM to DataFrame ===
results = (
    df_use["tool_description"]
    .apply(lambda desc: check_desc(desc).model_dump())
)
df_use[["relevant", "reason"]] = results.apply(pd.Series)

# Optional: inspect
print(df_use.head())

# === 7. Update use_aggregated column for the use case ===
def _to_list(cell: str) -> list[str]:
    """Convert a ;‑separated string (or NaN) → list without blanks."""
    if pd.isna(cell):
        return []
    return [u.strip() for u in str(cell).split(";") if u.strip()]

def update_use_aggregated(row):
    uses = _to_list(row["use_aggregated"])
    if row["relevant"]:
        if TARGET_USE not in uses:
            uses.append(TARGET_USE)
    else:
        uses = [u for u in uses if u != TARGET_USE]
    prefix = ";" if str(row["use_aggregated"]).startswith(";") else ""
    return prefix + ";".join(uses)

df_use["use_aggregated"] = df_use.apply(update_use_aggregated, axis=1)

# Save the updated DataFrame
df_use.to_excel(EXCEL_OUT, index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 354 entries, 53 to 19226
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   tool_id           354 non-null    int64 
 1   use_original      279 non-null    object
 2   use_aggregated    279 non-null    object
 3   name_tool         354 non-null    object
 4   tool_url_y        354 non-null    object
 5   tool_description  354 non-null    object
dtypes: int64(1), object(5)
memory usage: 19.4+ KB
{'relevant': False, 'reason': 'HireVue primarily focuses on digital interviewing and candidate screening, which are not directly related to compliance management.'}
     tool_id                                       use_original  \
4        778  ;payroll management;compliance management;cont...   
25      6748  ;cloud security & compliance;device management...   
111      726  ;data visualization;machine learning;complianc...   
144    13296  ;cloud applications & integratio

/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/6861976.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use[["relevant", "reason"]] = results.apply(pd.Series)
/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/6861976.py:99: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_use[["relevant", "reason"]] = results.apply(pd.Series)
/var/folders/lz/910twyk903nb6sxfgz54bngc0000gn/T/ipykernel_28883/6861976.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [ ]:
# Voorbeeld: Deep research prompt voor audio generation tools, met top 10 tools

from openai import OpenAI
client = OpenAI(timeout=3600)

# Stel de top 10 audio generation tools samen (voorbeeldnamen, vervang door echte top 10 indien beschikbaar)
top_10_audio_tools = [
    "Epidemic Sound",
    "Synthesia",
    "Cartesia",
    "Replica Studios",
    "Aiva",
    "Voicemod",
    "Descript",
    "LALAL.AI",
    "Murf AI",
    "Resemble AI"
]

tools_list_str = "\n".join([f"- {tool}" for tool in top_10_audio_tools])

input_text = f"""
Research the current landscape and future trends of audio generation AI tools, with a focus on the top 10 tools in the market:
{tools_list_str}

Do:
- Include specific figures, market size, growth rates, and notable use cases.
- For each of the top 10 tools, provide a brief summary of their product, business model, and any available usage or financial metrics.
- Prioritize reliable, up-to-date sources: peer-reviewed research, industry reports, tech news, or company financials.
- Include inline citations and return all source metadata.

Be analytical, avoid generalities, and ensure that each section supports
data-backed reasoning that could inform product strategy or investment decisions.
"""

response = client.responses.create(
  model="o4-mini-deep-research",
  input=input_text,
  tools=[
    {"type": "web_search_preview"},
    {"type": "code_interpreter", "container": {"type": "auto"}},
  ],
)

print(response.output_text)




# Market Overview & Trends  
The generative **audio AI** market is small but growing explosively. A recent report pegs the global *enterprise* audio generative AI market at about **$219.7M in 2024**, with an astonishing ~**43.9% CAGR** through 2030 ([www.grandviewresearch.com](https://www.grandviewresearch.com/horizon/statistics/enterprise-generative-ai-market/model-type/audio/global#:~:text=The%20global%20audio%20enterprise%20generative,from%202024%20to%202030)).  (More broadly, combined audio+video AI content generation is forecast to grow from ~$25.9B in 2024 to $132.6B by 2030 ([virtuemarketresearch.com](https://virtuemarketresearch.com/report/audio-or-visual-generative-ai-market#:~:text=,of%20possibilities%20for%20different%20competitors)) – underscoring rapid adoption in media and entertainment.) This growth is driven by demand for scalable content: from automated music and soundtracks to lifelike voiceovers in e-learning, gaming, advertising, and virtual assistants. Major AI lab

In [ ]:
## Analyzing duplicate tools issue

# Check how many unique tool_ids we have vs total entries
print("=== DUPLICATE ANALYSIS ===")
print(f"Total entries: {len(df_selected)}")
print(f"Unique tool_ids: {df_selected['tool_id'].nunique()}")
print(f"Duplicate entries: {len(df_selected) - df_selected['tool_id'].nunique()}")

# Find tools that appear multiple times
duplicate_tool_ids = df_selected['tool_id'].value_counts()
duplicates = duplicate_tool_ids[duplicate_tool_ids > 1]

print(f"\nNumber of tools that appear multiple times: {len(duplicates)}")
print(f"Most duplicated tool appears {duplicates.max()} times")

# Show examples of duplicated tools
print("\n=== EXAMPLES OF DUPLICATED TOOLS ===")
for tool_id in duplicates.head(5).index:
    print(f"\nTool ID {tool_id} appears {duplicates[tool_id]} times:")
    tool_rows = df_selected[df_selected['tool_id'] == tool_id]
    for idx, row in tool_rows.iterrows():
        print(f"  Row {idx}: {row['name_tool']} - use_original: {row['use_original']}")


In [ ]:
# Check if the duplicates have different use_original values (which would explain why they exist)
print("=== CHECKING WHY DUPLICATES EXIST ===")

# For each duplicated tool, check if the use_original values differ
for tool_id in duplicates.head(10).index:
    tool_rows = df_selected[df_selected['tool_id'] == tool_id]
    use_original_values = tool_rows['use_original'].dropna().unique()
    use_aggregated_values = tool_rows['use_aggregated'].dropna().unique()
    
    print(f"\nTool ID {tool_id} ({tool_rows.iloc[0]['name_tool']}):")
    print(f"  Different use_original values: {len(use_original_values)}")
    print(f"  Different use_aggregated values: {len(use_aggregated_values)}")
    
    if len(use_original_values) > 1:
        print(f"  use_original values: {list(use_original_values)}")
    if len(use_aggregated_values) > 1:
        print(f"  use_aggregated values: {list(use_aggregated_values)}")


In [ ]:
# Solution 1: Remove exact duplicates (same tool_id AND same use_original)
print("=== SOLUTION OPTIONS ===")

# Check for exact duplicates
exact_duplicates = df_selected.duplicated(subset=['tool_id', 'use_original'], keep='first')
print(f"Exact duplicates (same tool_id + use_original): {exact_duplicates.sum()}")

# Solution 1: Keep only unique combinations of tool_id + use_original
df_deduplicated = df_selected.drop_duplicates(subset=['tool_id', 'use_original'], keep='first')
print(f"After removing exact duplicates: {len(df_deduplicated)} entries")

# Solution 2: Keep only one record per tool_id (first occurrence)
df_one_per_tool = df_selected.drop_duplicates(subset=['tool_id'], keep='first')
print(f"After keeping only one record per tool: {len(df_one_per_tool)} entries")

# Solution 3: Aggregate multiple records per tool into one record
print("\n=== AGGREGATION APPROACH ===")
# Group by tool_id and combine use_original values
def combine_uses(series):
    # Remove NaN values and combine unique values
    values = series.dropna().astype(str)
    unique_values = []
    for val in values:
        # Split by semicolon and add to unique list
        for item in val.split(';'):
            item = item.strip()
            if item and item not in unique_values:
                unique_values.append(item)
    return ';'.join(unique_values)

df_aggregated = df_selected.groupby('tool_id').agg({
    'name_tool': 'first',
    'tool_url_y': 'first', 
    'tool_description': 'first',
    'use_original': combine_uses,
    'use_aggregated': combine_uses
}).reset_index()

print(f"After aggregating by tool_id: {len(df_aggregated)} entries")
print(f"This gives us exactly {df_selected['tool_id'].nunique()} unique tools")


In [35]:
# Select top 10 audio generation tools
# (Assume 'audio' is a boolean column indicating audio generation tools, as in previous cells)

# Use df_aggregated if defined, otherwise fall back to df_deduplicated
try:
    df_audio_source = df_aggregated
except NameError:
    print("df_aggregated is not defined, using df_deduplicated instead.")
    df_audio_source = df_deduplicated

if "audio" not in df_audio_source.columns:
    # If not present, try to infer from use_aggregated
    df_audio_source["audio"] = df_audio_source["use_aggregated"].str.contains("audio generation", case=False, na=False)

audio_tools = df_audio_source[df_audio_source["audio"] == True].copy()
top10_audio_tools = audio_tools.head(10)

# Prepare data for API call: convert to dicts (records)
audio_tools_payload = top10_audio_tools.to_dict(orient="records")

import requests

# Example API endpoint for o4-mini-deep-research (replace with actual endpoint if different)
api_url = "https://o4-mini-deep-research/api/tools"

# Make the API call (POST) with all variables for the top 10 audio generation tools
response = requests.post(api_url, json={"tools": audio_tools_payload})

# Print response status and content
print("API response status:", response.status_code)
try:
    print("API response content:", response.json())
except Exception as e:
    print("Could not decode JSON response:", e)
    print("Raw response content:", response.content)


df_aggregated is not defined, using df_deduplicated instead.


NameError: name 'df_deduplicated' is not defined